This notebook collects data from various sources and saves them in the database. <br>
The codes here could be executed to create the database for the first time.

Before starting the process, you would need MySQL. Install it to the local from [here](https://dev.mysql.com/downloads/mysql/). Remember the password since you would need them to connect to MySQL.

In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

print("Currently working at {}".format(os.getcwd()))

Currently working at /Users/gieunkwak/Data Analytics/platform


In [2]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

import investpy
from fredapi import Fred
import json
from matplotlib import pyplot as plt
# import plotly.express as px
# import plotly.graph_objects as go
import quandl
from datetime import date
from app.utils.utils import *
# from plotly.subplots import make_subplots

### Get credentials

In [4]:
with open('app/keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)
    
with open('app/utils/index_codes.json', 'r') as indx:
    index_codes = json.load(indx)

In [4]:
def fred_fred(code, idx_name, observation_start=None, observation_end=None):
    """
    date: 'MM/DD/YYY'
    """
    df = fred.get_series(code, observation_start=observation_start, observation_end=observation_end)
    df = pd.DataFrame(df).reset_index()
    df.columns = ['date', 'v']
    df.loc[:, 'code'] = code
    df.loc[:, 'code_name'] = idx_name
    
    df.loc[:, 'p_key'] = df['date'].astype(str).str.replace("-", "_") + "_" + df['code']
    return df
# fred.search('breakeven')

In [5]:
bei5y = fred_fred(index_codes['Fred']['Breakeven inflation rate 5y'], 
                 idx_name='BEI5y', observation_start='01/01/2012')
treasury10y = fred_fred(index_codes['Fred']['Treasury 10y'],
                    idx_name='tsry10y', observation_start='01/01/2012')

2022-05-15 17:33:31,022 [app.utils.utils] [INFO] Fetching data from fred: T5YIE, from 01/01/2012 to None.
2022-05-15 17:33:31,859 [app.utils.utils] [INFO] Fetching data from fred: DGS10, from 01/01/2012 to None.


In [6]:
treasury10y.head(2)

,date,v,code,code_name,p_key
0,2012-01-02,NaN,DGS10,tsry10y,2012_01_02_DGS10
1,2012-01-03,1.97,DGS10,tsry10y,2012_01_03_DGS10


In [84]:
def investing_api(call_type, ticker, from_date, to_date, country='united states'):
    """
    call_type: etf, stock, fund, index
    ticker: str. ticker name
    from_date: dd/mm/yyyy
    """
    if call_type == 'etf':
        # search name from ticker and return
        etfs = investpy.etfs.get_etfs(country=country)
        etf_name = etfs.loc[(etfs.symbol == ticker), 'name'].tolist()[0]
        data = investpy.get_etf_historical_data(etf=etf_name, country=country,
                                                from_date=from_date,
                                                to_date=to_date).reset_index()
    if call_type == 'stock':
        data = investpy.stocks.get_stock_historical_data(stock=ticker, country=country,
                                                         from_date=from_date,
                                                         to_date=to_date).reset_index()
    data.loc[:, 'ticker'] = ticker
    data.loc[:, 'p_key'] = data['Date'].astype(str).str.replace("-", "_") + "_" + data['ticker']
    return data

In [7]:
today = date.today().strftime("%m/%d/%Y")

In [8]:
today

'05/14/2022'

In [7]:
total_stock = investing_api('etf', 'VTI', '01/01/2020', '31/12/2020')

2022-05-15 17:34:53,484 [app.utils.utils] [INFO] Fetching etf from investing_api: VTI, from 01/01/2020 to 31/12/2020


In [8]:
total_stock.head(2)

,Date,Open,High,Low,Close,Volume,Currency,Exchange,ticker,p_key
0,2020-01-02,164.68,164.98,163.78,164.98,3291790,USD,NYSE,VTI,2020_01_02_VTI
1,2020-01-03,163.29,164.47,163.01,163.93,3625467,USD,NYSE,VTI,2020_01_03_VTI


## save them to local environment

In [2]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [48]:
def create_db_connection(host_name, user_name, user_password, db_name=None):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [46]:
connection = create_server_connection("localhost", "root", 'gi941505')

MySQL Database connection successful


In [47]:
create_database(connection, "CREATE DATABASE econ")

Database created successfully


connect to DB and create table

In [60]:
connection = create_db_connection("localhost", "root", 'gi941505', 'econ')

MySQL Database connection successful


In [79]:
create_fred_table = """
CREATE TABLE fred_daily (
    p_key VARCHAR(40) PRIMARY KEY,
    date DATE,
    v FLOAT(5),
    code VARCHAR(40) NOT NULL,
    code_name VARCHAR(40) NOT NULL
    );
"""

execute_query(connection, create_fred_table)
# execute_query(connection, 'DROP TABLE fred_daily') # delete table

Query successful


In [ ]:
create_stock_table = """
CREATE TABLE stock_daily (
    p_key VARCHAR(40) PRIMARY KEY,
    date DATE,
    v FLOAT(5),
    code VARCHAR(40) NOT NULL,
    code_name VARCHAR(40) NOT NULL
    );
"""

execute_query(connection, create_stock_table)
# execute_query(connection, 'DROP TABLE fred_daily') # delete table

In [87]:
total_stock.head(2)

,Date,Open,High,Low,Close,Volume,Currency,Exchange,ticker,p_key
0,2020-01-02,164.68,164.98,163.78,164.98,3291790,USD,NYSE,VTI,2020_01_02_VTI
1,2020-01-03,163.29,164.47,163.01,163.93,3625467,USD,NYSE,VTI,2020_01_03_VTI


In [72]:
bei5y.head(2)

,date,v,code,code_name,p_key
0,2012-01-02,NaN,T5YIE,BEI5y,2012_01_02_T5YIE
1,2012-01-03,1.66,T5YIE,BEI5y,2012_01_03_T5YIE


In [ ]:
create_teacher_table = """
CREATE TABLE teacher (
  teacher_id INT PRIMARY KEY,
  first_name VARCHAR(40) NOT NULL,
  last_name VARCHAR(40) NOT NULL,
  language_1 VARCHAR(3) NOT NULL,
  language_2 VARCHAR(3),
  dob DATE,
  tax_id INT UNIQUE,
  phone_no VARCHAR(20)
  );
 """

connection = create_db_connection("localhost", "root", pw, db) # Connect to the Database
execute_query(connection, create_teacher_table) # Execute our defined query

In [ ]:
https://www.freecodecamp.org/news/connect-python-with-sql/

In [ ]:
CREATE TABLE client (
  client_id INT PRIMARY KEY,
  client_name VARCHAR(40) NOT NULL,
  address VARCHAR(60) NOT NULL,
  industry VARCHAR(20)
);